# Microsoft-Vision-ResNet50 based Hybrid Model using Linear Classifier

In [ ]:
#conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

#pip install progressbar2


In [ ]:

# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier/'
assert FOLDERNAME is not None, "[!] Enter the foldername."


import sys
sys.path.append('/home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier')

%cd /$FOLDERNAME/datasets/
!bash get_datasets.sh
%cd /$FOLDERNAME

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from data_utils import get_CIFAR10_data

%load_ext autoreload
%autoreload 2

def calculate_error(x, y):
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

import progressbar
from progressbar import progressbar

from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.linear_model import LogisticRegression

In [ ]:
import microsoftvision

In [ ]:
class Wrangling:
    def __init__(self):
        self.wrangling = transforms.Compose([
                                           transforms.Resize(224),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])])

    def __call__(self, x):
        return self.wrangling(x)[[2,1,0],:,:]

In [ ]:

data = get_CIFAR10_data()
for k, v in list(data.items()):
    print(f"{k}: {v.shape}")

In [ ]:


train = CIFAR10('./path', download=True, train=True, transform=Wrangling())
test = CIFAR10('./path', download=True, train=False, transform=Wrangling())

In [ ]:
model = microsoftvision.models.resnet50(pretrained=True)

In [ ]:
model.eval()
model.cuda()

In [ ]:
def extractor(dataset, model):
    all_features = []
    all_labels = []
    import time
    import logging
    import progressbar
    torch.cuda.empty_cache()

    with torch.no_grad():
        for point, labels in progressbar.progressbar(DataLoader(dataset, batch_size=128, num_workers=8)):
            point = point.cuda()
            labels = labels.cuda()
            features = model(point)

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [ ]:
train_features, train_labels = extractor(train, model)
test_features, test_labels = extractor(test, model)

In [ ]:
classifier = LogisticRegression(random_state=0, max_iter=1000, verbose=1, n_jobs=16)

In [ ]:
classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)
result = 100 * np.mean((test_labels == predictions).astype(np.float))
print(f"Hybrid Model Performance Accuracy: {result}")